<a href="https://colab.research.google.com/github/psygrammer/psypy/blob/master/notebooks/03_b_funcions_and_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. 내장 자료구조, 함수, 파일 (2)

* 싸이그래머 / PsyPy [1]
* 김무성

In [0]:
!python -V

Python 3.6.8


# 차례 
* 함수
    - 네임스페이스, 스코프, 지역 함수
    - 여러 값 반환하기
    - 함수도 객체다
    - 익명 함수
    - 클로저 : 함수를 반환하는 함수
    - \*args와  \*\*kwargs를 사용해서 호출 문법 확장하기  
    - 커링 : 일부 인자만 취하기
    - 제너레이터
    - 에러와 예외 다루기
* 파일과 운영체제
    - 바이트와 유니코드 파일
* 클래스
* 함수형 프로그래밍
  - map
  - filter
  - reduce

------------------------

# 함수

```python
def my_function(x, y, z=1.5) :
    if z > 1 :
        return z * (x + y)
    else :
        return z / (x + y)
    
# return문이 몇 개가 되든 상관없다. 
# 함수 블록이 끝날 때까지 return문이 없다면 None이 반환된다.   
```

## 네임스페이스, 스코프, 지역 함수

```python
# 로컬
def func() :
    a = []  # <- local
    for i in range(5) :
        a.append(i)
```        

```python
# 전역 
a = []
def func() :
    for i in range(5) :
        a.append(i)
        
func()
a
```

In [0]:
b = None
def bind_b_variable() :
    b = []
    
bind_b_variable()    
print(b)    

None


In [0]:
# global 예약어 사용
b = None
def bind_b_variable() :
    global b
    b = []

bind_b_variable()
print(b)

[]


```python
# 함수 안에서 함수를 선언할 수 있다.
def outer_function(x, y, z) :
    def inner_function(a, b, c) :
        pass
    pass
```

## 여러 값 반환하기

In [0]:
def f() :
    a = 5
    b = 6
    c = 7
    return a, b, c

a, b, c = f()
print(a, b, c)

5 6 7


In [0]:
return_value = f()
return_value # 여래 개의 값을 하나의 변수에 반환받으면, 그 변수는 튜플 자료형이 된다.

(5, 6, 7)

## 함수도 객체다

In [0]:
states = [ '  Alabama ', 'Georgia!', 'Georgia', 'georgia', 'FlOrIda', 
          'south carolin##', 'West virginaia?']

### 문자열을 변형하기 위해 함수를 사용하는 일반적인 패턴

In [0]:
import re 

def clean_strings(strings) :
    result = []
    for value in strings :
        value = value.strip()
        value = re.sub('[!#?]', '', value) # 문장 부호 제거
        value = value.title()
        result.append(value)
    return result

In [0]:
clean_strings(states)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South Carolin',
 'West Virginaia']

### 함수를 객체화하고 인자로 넘겨서 좀 더 유연하게 사용하는 패턴

In [0]:
def remove_punctuation(value) :
    return re.sub('[!#?]', '', value)

# 함수도 객체. 
clean_ops = [str.strip, remove_punctuation, str.title]

def clean_strings(strings, ops) :
    result = []
    for value in strings :
        for function in ops :
            value = function(value) 
        result.append(value)
    return result

In [0]:
clean_strings(states, clean_ops)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South Carolin',
 'West Virginaia']

In [0]:
# map을 이용해본다. 어떤 컬렉션에 대해 함수를 적용하는 함수다. 
# 이를 이용해서 함수를 인자로 사용할 수 있다.
map(remove_punctuation, states)

In [0]:
for e in map(remove_punctuation, states) :
    print(e)

  Alabama 
Georgia
Georgia
georgia
FlOrIda
south carolin
West virginaia


## 익명 함수 

```python
def short_function(x) :
    return x * 2

# 위의 함수와 동치인 익명 함수(람다 함수)
equiv_anon = lambda x: x * 2
```

In [0]:
# 각 문자열에서 다양한 문자가 포함된 순서로 정렬하는 예제
strings = ['foo', 'card', 'bar', 'aaaa', 'abab']
strings.sort(key=lambda x: len(set(list(x))))
strings

['aaaa', 'foo', 'abab', 'bar', 'card']

## 클로저 : 함수를 반환하는 함수

In [0]:
# 클로저는 다른 함수에서 반환되는, 동적으로 생성된 함수다. 
# 핵심은 반환되는 함수는 그 함수가 생성된 시점의 
# 지역 네임스페이스의 변수에 접근할 수 있다는 점이다.

In [0]:
def make_closure(a) :
    def closure() :
        print("I know the secret: %d", a)
    return closure

closure = make_closure(5)
closure()
closure() # 클로저를 생성하는 함수가 끝나더라도 생성된 시점의 네임스페이스에 여전히 접근할 수 있다.


In [0]:
# 클로져가 변경 가능한 객체를 가질 수도 있다.
def make_watcher() :
    have_seen = {}
    
    def has_been_seen(x) :
        if x in have_seen :
            return True
        else :
            have_seen[x] = True
            return False
        
    return has_been_seen

In [0]:
watcher = make_watcher()
vals = [5, 6, 1, 5, 1, 6, 3, 5]
[watcher(x) for x in vals]

## \*args와  \*\*kwargs를 사용해서 호출 문법 확장하기  

```python
# 파이썬에서 함수 인자가 동작하는 방식은, 
# 만약 func(a, b, c, d=some, e= value) 라고 코드를 작성하면
# 일반 인자와 키워드 인자는 
# 각각 튜플 args와
# 사전 kwargs에 저장된다.
# 내부적으로 다음과 같은 과정을 거친다
a, b, c = args
d = kwargs.get('d', d_default_value)
e = kwargs.get('e', e_defaulT_value)
```

In [0]:
# 예제
def say_hello_then_call_f(f, *args, **kwargs) :
    print('args is', args)
    print('kwargs is', kwargs)
    print("Hello! Now I'm going to call %s" %(f))
    return f(*args, **kwargs)

def g(x, y, z=1) :
    return (x+y)/z

say_hello_then_call_f(g, 1, 2, z=5.)

## 커링 : 일부 인자만 취하기

In [0]:
# 함수에서 일부 인자를 고정해 새로운 함수를 만드는 기법을 커링이라고 한다.

def add_numbers(x, y) :
    return x + y

# 여기서 add_numbers의 두 번째 인자가 '커리되었다'라고 한다.
add_five = lambda y: add_numbers(5, y) 


# 다음 방식으로 더 단순하게 만들 수 있다.
from functools import partial
add_five = partial(add_numbers, 5)

## 제너레이터

* 이터레이터 프로토콜
* 제너레이터
* 제너레이터 표현식
* itertoos 모듈

### 이터레이터 프로토콜

In [0]:
# 파이썬은 리스트 내의 객체나 파일의 각 행 같은 순차적인 자료를 순회하는 일관적인 방법을 제공한다.
# 이터레이터 프로토콜을 이용해서 순회가 가능한 객체를 만들 수 있다.

some_dict = {'a':1, 'b':2, 'c':3}
 
# for key in some_dict 라고 작성하면 
# 파이썬 인터프리터는 some_dict에서 이터레이터를 생성한다.    
for key in some_dict :
    print(key)

In [0]:
list(dict_iterator)

### 제너레이터

In [0]:
# 제너레이터는 순회가 가능한 객체를 생성하는 간단한 방법이다.
# 일반 함수는 실행되면 단일 값을 반환하지만
# 제너레이터는 순차적인 값을 매 요청 시마다 하나씩 반환한다.
# 제너레이터를 생성하려면 함수에서 return 대신에 yeild 예약어를 사용한다.
def squares(n=10) :
    print('Generating squares from 1 to %d' %(n ** 2))
    for i in range(1, n+1) :
        yield i ** 2

In [0]:
gen = squares()
gen

In [0]:
for x in gen :
    print(x) 

### 제너레이터 표현식

In [0]:
# 제너레이터를 만드는 간단한 방법은 제너레이터 표현식을 사용하는 것이다.
gen = (x ** 2 for x in range(100))
gen

In [0]:
# 위의 코드는 다음 코드와 동일하다
def _make_gen() :
    for x in range(100) :
        yield x ** 2
        
gen = _make_gen()
gen

In [0]:
# 예제
max(x ** 2 for x in range(100))

In [0]:
dict((i, i**2) for i in range(5))

### itertools 모듈

In [0]:
# 일반 데이터 알고리즘을 위한 제너레이터를 포함
import itertools

# 예를 들어 groupby는 순차 자료 구조와 함수를 인자로 받아,
# 인자로 받은 함수에서 반환하는 값에 따라 그룹을 지어준다.
first_letter = lambda x : x[0]

names = ['Alan', 'Adam', 'Wes', 'Will', 'Albert', 'Steven']

for letter, names in itertools.groupby(names, first_letter) :
    print(letter, list(names))


---------------------------------

# 파일과 운영체제
* 파일 읽기
* with 문 사용
* 파일 쓰기
* 바이트와 유니코드

------------------------------

실습할 파일을 준비.(웹에서 가져오기 - urllib 사용)

In [0]:
# 우선 파일을 가져와보자.

## 1. 디렉토리 만들기
import os

dir_name = "examples"
if not os.path.exists(dir_name) :
  os.mkdir(dir_name)

In [28]:
# example 디렉토리가 생겼다.
%ls

examples/  sample_data/


In [0]:
# 지금은 디렉토리 안에 아무 파일도 없다.
%ls examples/

In [0]:
# 2. 파일 가져오기 - [1]의 예제소스 github의 파일들을 주소로 접근해서 가져온다.

import urllib.request

url = 'https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/segismundo.txt'
file_name = "examples/segismundo.txt" 
with urllib.request.urlopen(url) as r:
  doc = r.read()
  with open(file_name, "wb") as f:
    f.write(doc)

In [36]:
# 다운로드 완료. 파일이 생겼다.
%ls examples/

segismundo.txt


In [37]:
# 내용를 한번 살펴보자 
!head examples/segismundo.txt

Sueña el rico en su riqueza,
que más cuidados le ofrece;

sueña el pobre que padece
su miseria y su pobreza;

sueña el que a medrar empieza,
sueña el que afana y pretende,
sueña el que agravia y ofende,



--------------

## 파일 읽기

In [0]:
파일을 읽고 쓰기 위해서는 내장 함수인 open을 이용해서 파일의 상대 경로나 절대 경로를 넘겨주어야 한다.

In [0]:
path = "examples/segismundo.txt"

In [0]:
f = open(path)

기본적으로 파일은 읽기 전용 모드인 'r'로 열린다. 파일 핸들 f를 리스트로 생각할 수 있으며 파일의 매 줄을 순회할 수 있다.

In [42]:
for line in f:
  print(line) # 여기서 줄별로 처리할 코드를 넣으면 된다.

Sueña el rico en su riqueza,

que más cuidados le ofrece;



sueña el pobre que padece

su miseria y su pobreza;



sueña el que a medrar empieza,

sueña el que afana y pretende,

sueña el que agravia y ofende,



y en el mundo, en conclusión,

todos sueñan lo que son,

aunque ninguno lo entiende.





파일에서 읽은 줄은 줄끝(end-of-line, EOL) 문자가 그대로 남아 있으므로 파일에서 ㅇ릭은 줄에서 이를 제거하는 다음과 같은 코드를 많이 사용한다.(string의 rstrip() 혹은 strip() 코드를 적용)

In [0]:
lines = [x.rstrip() for x in open(path)]

In [44]:
lines

['Sueña el rico en su riqueza,',
 'que más cuidados le ofrece;',
 '',
 'sueña el pobre que padece',
 'su miseria y su pobreza;',
 '',
 'sueña el que a medrar empieza,',
 'sueña el que afana y pretende,',
 'sueña el que agravia y ofende,',
 '',
 'y en el mundo, en conclusión,',
 'todos sueñan lo que son,',
 'aunque ninguno lo entiende.',
 '']

파일 객체를 생성하기 위해 open을 사용했다면 작업이 끝났을 때 명시적으로 닫아주어야 한다. 파일을 닫으면 해당 자원을 운영체제로 되돌려준다.

In [0]:
f.close()

In [46]:
# 위의 코드들을 한번에 쓰면 다음과 같다. 
path = "examples/segismundo.txt"
f = open(path)
for line in f:
  lines = [x.strip() for x in open(path)] # 오른쪽 끝만 정리하는 rstrip 대신에 줄의 양쪽 끝을 다 정리하는 strip 함수 사용

lines

['Sueña el rico en su riqueza,',
 'que más cuidados le ofrece;',
 '',
 'sueña el pobre que padece',
 'su miseria y su pobreza;',
 '',
 'sueña el que a medrar empieza,',
 'sueña el que afana y pretende,',
 'sueña el que agravia y ofende,',
 '',
 'y en el mundo, en conclusión,',
 'todos sueñan lo que son,',
 'aunque ninguno lo entiende.',
 '']

## With 문 사용

with 문을 사용하면 파일 작업이 끝날을 때 필요한 작업을 쉽게 처리할 수 있다.

In [49]:
# 이렇게 하면 with 블록이 끝나는 시점에 파일 핸들 f를 자동으로 닫아준다.  
with open(path) as f:
  lines = [x.strip() for x in f]

lines

['Sueña el rico en su riqueza,',
 'que más cuidados le ofrece;',
 '',
 'sueña el pobre que padece',
 'su miseria y su pobreza;',
 '',
 'sueña el que a medrar empieza,',
 'sueña el que afana y pretende,',
 'sueña el que agravia y ofende,',
 '',
 'y en el mundo, en conclusión,',
 'todos sueñan lo que son,',
 'aunque ninguno lo entiende.',
 '']

# 클래스 [3]

In [0]:
# 클래스를 하나 만들기 위해 특정 객체의 하위 클래스를 만들 수 있습니다.
class Human(object):

    # 클래스 속성은 이 클래스의 모든 인스턴스에서 공유합니다.
    species = "H. sapiens"

    # 기본 초기화자
    def __init__(self, name):
        # 인자를 인스턴스의 name 속성에 할당합니다.
        self.name = name

    # 모든 인스턴스 메서드에서는 self를 첫 번째 인자로 받습니다.
    def say(self, msg):
       return "%s: %s" % (self.name, msg)

    # 클래스 메서드는 모든 인스턴스에서 공유합니다.
    # 클래스 메서드는 호출하는 클래스를 첫 번째 인자로 호출됩니다.
    @classmethod
    def get_species(cls):
        return cls.species

    # 정적 메서드는 클래스나 인스턴스 참조 없이도 호출할 수 있습니다.
    @staticmethod
    def grunt():
        return "*grunt*"

In [2]:
# 클래스 인스턴스화
i = Human(name="Ian")
print(i.say("hi"))     

Ian: hi


In [3]:
j = Human("Joel")
print(j.say("hello"))

Joel: hello


In [4]:
# 클래스 메서드를 호출
i.get_species() 

'H. sapiens'

In [5]:
# 공유 속성을 변경
Human.species = "H. neanderthalensis"
print(i.get_species()) 
print(j.get_species()) 

H. neanderthalensis
H. neanderthalensis


In [6]:
# 정적 메서드를 호출
Human.grunt()

'*grunt*'

In [0]:
class HousePark:
    lastname = "박"
    def __init__(self, name):
        self.fullname = self.lastname + name
    def travel(self, where):
        print("%s, %s여행을 가다." % (self.fullname, where))
    def love(self, other):
        print("%s, %s 사랑에 빠졌네" % (self.fullname, other.fullname))
    def fight(self, other):
        print("%s, %s 싸우네" % (self.fullname, other.fullname))
    def __add__(self, other):
        print("%s, %s 결혼했네" % (self.fullname, other.fullname))
    def __sub__(self, other):
        print("%s, %s 이혼했네" % (self.fullname, other.fullname))
    def __del__(self):
        print("%s 죽네" % self.fullname)

class HouseKim(HousePark):
    lastname = "김"
    def travel(self, where, day):
        print("%s, %s여행 %d일 가네." % (self.fullname, where, day))

In [0]:
pey = HousePark("응용")
juliet = HouseKim("줄리엣")

In [9]:
pey.travel("부산")
juliet.travel("부산", 3)
pey.love(juliet)
pey + juliet
pey.fight(juliet)
pey - juliet

박응용, 부산여행을 가다.
김줄리엣, 부산여행 3일 가네.
박응용, 김줄리엣 사랑에 빠졌네
박응용, 김줄리엣 결혼했네
박응용, 김줄리엣 싸우네
박응용, 김줄리엣 이혼했네


In [11]:
pey.__del__()

박응용 죽네


--------------------

# 함수형 프로그래밍 [4]

* map
* filter
* reduce

## map

map이라는 것은 함수와 시퀀스 자료형(리스트, 터플, 문자열)을 입력으로 받아서 시퀀스 자료형의 각각의 요소가 함수의 입력으로 들어간 다음 나오는 출력값을 묶어서 리스트로 돌려주는 함수이다.

In [12]:
def f(x) :
    return x*x

x = [1, 2, 3, 4, 5]
y = map(f, x)

print(x)
print(list(y))

[1, 2, 3, 4, 5]
[1, 4, 9, 16, 25]


<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.7.png?raw=1'/>

In [13]:
# 위의 코드는 다음과 같다
y = []
for e in x :
    ne = f(e)
    y.append(ne)

print(x)
print(y)

[1, 2, 3, 4, 5]
[1, 4, 9, 16, 25]


In [14]:
# 람다 함수 이용
y = map(lambda a:a*a, x)
print(x)
print(list(y))

[1, 2, 3, 4, 5]
[1, 4, 9, 16, 25]


In [15]:
x = range(10)
y = map(lambda a: a*a+4*a, x)
print(x)
print(list(y))

range(0, 10)
[0, 5, 12, 21, 32, 45, 60, 77, 96, 117]


In [16]:
# 두 개 이상 입력
x = [1,2,3,4,5]
y = [6,7,8,9,10]
z = map(lambda a, b:a+b, x, y)
print(x)
print(y)
print(list(z))

[1, 2, 3, 4, 5]
[6, 7, 8, 9, 10]
[7, 9, 11, 13, 15]


<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.8.png?raw=1'/>

## filter

filter는 함수와 시퀀스 자료형을 입력으로 받아서 자료의 값이 하나씩 함수의 인수로 전달될 때, 참을 반환시키는 값만을 따로 모아서 리스트의 형태로 반환하는 함수이다. filter의 뜻은 무엇인가를 걸러낸다는 뜻이다. 이 의미가 filter 함수에서도 그대로 사용된다.

In [17]:
def positive(x): 
    return x > 0

list(filter(positive, [1,-3,2,0,-5,6]))

[1, 2, 6]

In [18]:
# 위의 코드는 다음과 같다
def positive(l): 
    result = [] 
    for i in l: 
        if i > 0: 
            result.append(i) 
    return result

positive([1,-3,2,0,-5,6])

[1, 2, 6]

In [19]:
# 람다 함수 사용
list(filter(lambda x : x>0, [1,-3,2,0,-5,6]))

[1, 2, 6]

In [20]:
list(filter(lambda x : x%2, [1,2,3,4,5,6]))

[1, 3, 5]

<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.11.png?raw=1'/>

## reduce

reduce는 첫 인수로 함수를 받는다. 이 함수는 두 개의 인수를 받는다. 두 번째 인자는 시퀀스 자료형이다. 시퀀스의 자료들은 순차적으로 reduce가 받은 함수의 첫 번째 인수, 두 번째 인수로 전달된다. 첫 인수는 함수의 계산 결과가 누적적으로 적용된다. 단, 처음 계산에서는 시퀀스의 두 개의 요소가 함수에 전달된다.

In [0]:
from functools import reduce

In [22]:
# 1부터 5까지 더하기
# 다음 코드는 이 수식과 동일하다. ((((1+2)+3)+4)+5)
reduce(lambda x, y : x+y, [1, 2, 3, 4, 5])

15

<img src='https://github.com/psygrammer/psypy/blob/master/notebooks/figures/03/01_fig.10.12.png?raw=1'/>

In [25]:
# reduce 함수에 세 번째 인수를 부여할 수 있는데, 초기 값으로 사용된다.
# 다음 코드는 이 수식과 동일하다. (((((0+1)+2)+3)+4)+5)
reduce(lambda x, y : x+y, [1, 2, 3, 4, 5], 0)

15

In [26]:
# 각 원소를 제곱해서 모두 더하기
reduce(lambda x, y : x + y*y, range(1,11), 0)

385

In [24]:
# 위의 코드는 다음과 동일하다.
x = 0
for y in range(1, 11) :
    x = x + y*y
    
x

385

-------------------------------

# 참고자료 
* [1] 파이썬 라이브러리를 활용한 데이터 분석(2판)
    - http://www.hanbit.co.kr/store/books/look.php?p_code=B6417848794
* [2] 점프 투 파이썬 : 04장 프로그램의 입력과 출력은 어떻게 해야 할까? - https://wikidocs.net/23
* [3] 점프 투 파이썬 : 클래스 - https://wikidocs.net/28
* [4] 점프 투 파이썬 : 내장함수 - https://wikidocs.net/32